<a href="https://colab.research.google.com/github/TiantianWang-Sara/Machine-Learning-Projects/blob/main/Project_2_LSMC_with_JAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Due date: October 31 2024

# Description
  In this problem we will use apply the LSMC method to price American put options. Specifically, we will replicate the result in the first row, 6th column of Table 1 in [Longstaff and Schwartz 2001](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf)

  

*  Read the introduction of the [paper](https://www.anderson.ucla.edu/documents/areas/fac/finance/least_squares.pdf).
*   We will price an american put option as described in page 126 of the aforementioned article. Read paragraphs 1 and 2 of page 126
* As we saw in class, one of the ways we can use linear regression to fit nonlinear functions is to use polynomial features. A common choice in many applications is to use the so called ``Chebyshev polynomials''. Chebyshev polynomials are defined recursively by:

\begin{equation}
T_0(x) = 1\\
T_1(x) = x\\
T_{n + 1}(x)  =  2 x T_n(x) - T_{n - 1}(x)\\
\end{equation}


# Part 1
The code below simulates the evolution of a stock price that follows a geometric brownian motion. Write a JAX version of that code. You are not allowed to use functions from other libraries. For this part, the "simulate"
function does not need to be jit compiled. As we will see, jit compiling a funciton with for loops may introduce some complications


In [ ]:
import numpy as np

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100


def simulate():
  np.random.seed(0)

  def step(S):
    dZ = np.random.normal(size=S.size) * np.sqrt(dt)
    dS = r * S  * dt + σ  * S  * dZ
    S = S + dS
    return S

  S0 = np.ones(20000)
  S = S0
  S_list = []

  for t in range(m):
    S = step(S)
    S_list.append(S)

  S_array = np.stack(S_list)
  return S_array

simulate()

array([[1.00715621, 1.00170063, 1.00401495, ..., 0.99987096, 0.99586427,
        0.99879389],
       [1.00858656, 1.00179888, 1.00740096, ..., 1.00672951, 0.99676042,
        0.99400235],
       [1.0008628 , 1.00485977, 1.01312172, ..., 1.01043822, 0.98842005,
        0.99337036],
       ...,
       [1.00834903, 0.96949369, 0.96891127, ..., 1.06609344, 0.98978317,
        1.00386936],
       [1.0045846 , 0.97349649, 0.96590568, ..., 1.06415365, 0.98488869,
        1.00491575],
       [1.00652461, 0.97005164, 0.96454623, ..., 1.05651341, 0.98368471,
        1.00345296]])

In [ ]:
import jax.numpy as jnp
import jax

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100

@jax.jit
def simulate():
  key = jax.random.key(0)

  def step(S):
    dZ = jax.random.normal(subkey, shape = S.shape) * jnp.sqrt(dt)
    dS = r * S  * dt + σ  * S  * dZ
    S = S + dS
    return S

  S0 = jnp.ones(20000)
  S = S0
  S_list = []

  for t in range(m):
    key, subkey = jax.random.split(key)
    S = step(S)
    S_list.append(S)

  S_array = jnp.stack(S_list)
  return S_array

simulate()

Array([[0.9990368 , 0.99845755, 1.0009075 , ..., 0.9949413 , 0.9968931 ,
        0.99635816],
       [0.99916214, 0.99233675, 0.9988377 , ..., 0.9934248 , 0.9962843 ,
        1.0000815 ],
       [1.0038999 , 0.99782735, 0.9933294 , ..., 0.9843731 , 0.99664205,
        0.99447113],
       ...,
       [0.98081917, 1.0411566 , 0.98559856, ..., 1.0382012 , 1.0340495 ,
        0.9722904 ],
       [0.9760958 , 1.0460585 , 0.9811195 , ..., 1.0345007 , 1.0348959 ,
        0.977704  ],
       [0.97404456, 1.0439519 , 0.9716244 , ..., 1.0376757 , 1.034841  ,
        0.974258  ]], dtype=float32)

# Part 2
Write a jit compiled version of the simulate function. You may want to check out the function jax.lax.scan.


In [ ]:
import jax.numpy as jnp
import jax

# Data
σ = 0.04
r = 0.01
K = 35

# Design choice
dt = 0.01
m = 100

@jax.jit
def simulate():
  key = jax.random.key(0)
  key = jax.random.split(key,m)

  def step(S,key):
    dZ = jax.random.normal(key, shape = S.shape) * jnp.sqrt(dt)
    dS = r * S  * dt + σ  * S  * dZ
    S = S + dS
    return S, S

  S0 = jnp.ones(20000)
  S = S0
  S_list = []

  S, S_list = jax.lax.scan(step, S, key)

  return S_list

simulate()

Array([[0.9933954 , 1.0010924 , 1.0007713 , ..., 1.001979  , 0.99986166,
        1.0042802 ],
       [0.99128497, 1.0050147 , 1.0016878 , ..., 1.0015069 , 0.9999063 ,
        1.0138991 ],
       [0.98903215, 1.0027052 , 1.0014749 , ..., 0.996598  , 1.0071341 ,
        1.0048819 ],
       ...,
       [1.0297807 , 1.0876848 , 1.0172694 , ..., 1.0528882 , 0.9786827 ,
        1.0096897 ],
       [1.024948  , 1.0880603 , 1.0160227 , ..., 1.05423   , 0.9789505 ,
        1.0111402 ],
       [1.026088  , 1.0818837 , 1.0110286 , ..., 1.0537153 , 0.98459226,
        1.0074184 ]], dtype=float32)

# Part 3
The code below computes the price of an American Put option using Least Squares Monte Carlo (LSMC). Write a JAX version of that code. You are not allowed to use functions from other libraries. Your "compute_price" function must be jit compiled.

In [ ]:
import numpy as np

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 12   # Polynmial order
Δt = T / m  # interval between two exercise dates


# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x, k):
    B = [np.ones(len(x)), x]
    for n in range(2, k):
        Bn = 2 * x * B[n - 1] - B[n - 2]
        B.append(Bn)

    return np.column_stack(B)


# scales x to be in the interval(-1, 1)
def scale(x):
    xmin = x.min()
    xmax = x.max()
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b


# simulates one step of the stock price evolution
def step(S):
    dB = np.sqrt(Δt) * np.random.normal(size=S.size)
    S_tp1 = S + r * S * Δt + σ * S * dB
    return S_tp1


def payoff_put(S):
    return np.maximum(K - S, 0.)


# LSMC algorithm
def compute_price():
    np.random.seed(0)
    S0 = Spot * np.ones(n)
    S = [S0]

    for t in range(m):
        S_tp1 = step(S[t])
        S.append(S_tp1)

    discount = np.exp(-r * Δt)

    # Very last date
    value_if_exercise = payoff_put(S[-1])
    discounted_future_cashflows = value_if_exercise * discount

    # Proceed recursively
    for i in range(m - 1):
        X = chebyshev_basis(scale(S[-2 - i]), order)
        Y = discounted_future_cashflows

        Θ = np.linalg.solve(X.T @ X, X.T @ Y)
        value_if_wait = X @ Θ
        value_if_exercise = payoff_put(S[-2 - i])
        exercise = value_if_exercise >= value_if_wait
        discounted_future_cashflows = discount * np.where(
            exercise,
            value_if_exercise,
            discounted_future_cashflows)

    return discounted_future_cashflows.mean()


print(compute_price())
# test = compute_price(order, Spot, σ, K, r)


4.4634476148734405


In [ ]:
import jax.numpy as jnp
import jax

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 100000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 12   # Polynmial order
Δt = T / m  # interval between two exercise dates

# Construct polynomial features of order up to k using the
# recursive formulation
def chebyshev_basis(x, k):
    B = [jnp.ones(len(x)), x]
    for n in range(2, k):
        Bn = 2 * x * B[n - 1] - B[n - 2]
        B.append(Bn)

    return jnp.column_stack(B)


# scales x to be in the interval(-1, 1)
def scale(x):
    xmin = x.min()
    xmax = x.max()
    a = 2 / (xmax - xmin)
    b = 1 - a * xmax
    return a * x + b


# simulates one step of the stock price evolution
def step(S, key):
    dB = jax.random.normal(key, shape = S.shape) * jnp.sqrt(Δt)
    S_tp1 = S + r * S * Δt + σ * S * dB
    return S_tp1, S_tp1


def payoff_put(S):
    return jnp.maximum(K - S, 0.)


# LSMC algorithm
@jax.jit
def compute_price():
    key = jax.random.key(0)
    key = jax.random.split(key,m)
    S0 = Spot * jnp.ones(n)
    S = [S0]
    S = jnp.stack(S)
    S_tp1, S_stack = jax.lax.scan(step, S[0], key)

    S = jnp.vstack((S,S_stack))

    discount = jnp.exp(-r * Δt)

    # Very last date
    value_if_exercise = payoff_put(S[-1])
    discounted_future_cashflows = value_if_exercise * discount

    def value(discounted_future_cashflows, i):
        X = chebyshev_basis(scale(S[-2 - i]), order)
        Y = discounted_future_cashflows

        Θ = jnp.linalg.solve(X.T @ X, X.T @ Y)
        value_if_wait = X @ Θ
        value_if_exercise = payoff_put(S[-2 - i])
        exercise = value_if_exercise >= value_if_wait
        discounted_future_cashflows = discount * jnp.where(
            exercise,
            value_if_exercise,
            discounted_future_cashflows)
        return discounted_future_cashflows, discounted_future_cashflows

    discounted_future_cashflows, discounted_future_cashflows_stack = jax.lax.scan(value, discounted_future_cashflows, jnp.arange(m-1))

    return discounted_future_cashflows.mean()


print(compute_price())
# test = compute_price(order, Spot, σ, K, r

4.4776163
